In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("C:/Users/vladi/Documents/stocks_entry_prediction")
sys.path.append("C:/Users/vladi/Documents/stocks_entry_prediction/vtb_scorekit")

from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import ta
# from datetime import datetime as dt
from sklearn.preprocessing import FunctionTransformer
from catboost import CatBoostClassifier
import warnings

from main.calc_features import calc_features

warnings.filterwarnings("ignore")

In [2]:
TEST_SAVE_NAME = "predicted_2024_03.csv"

In [3]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x/period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x/period * 2 * np.pi))

In [4]:
# df = pd.read_csv("../src/daily_data/Si_230101_240831.csv", sep=";")
df = pd.read_excel("../src/stock_data.xlsx", sheet_name="Si")

In [5]:
df = calc_features(df, "Si", 200, 400)

In [6]:
df.head()

,OPEN,HIGH,LOW,CLOSE,VOL,DATETIME,WEEK_DAY,HOUR,CANDLE_COLOR,PIVOT,...,END_CORRECTION_PERC_MA_mean,END_CORRECTION_PERC_MA_std,UPGOING_ACTIVE_IMPULSE_MA_mean,UPGOING_ACTIVE_IMPULSE_MA_std,DOWNGOING_ACTIVE_IMPULSE_MA_mean,DOWNGOING_ACTIVE_IMPULSE_MA_std,IS_SUPERACTIVE_IMPULSE_MA_mean,IS_SUPERACTIVE_IMPULSE_MA_std,TARGET_LONG,TARGET_SHORT
0,70105.0,70500.0,70105.0,70378.0,17203.0,2023-01-03 09:00:00,1,9,green,70327.664062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,70383.0,70392.0,70205.0,70286.0,5659.0,2023-01-03 09:05:00,1,9,red,70294.335938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,70282.0,70360.0,70163.0,70296.0,4963.0,2023-01-03 09:10:00,1,9,green,70273.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,70298.0,70350.0,70233.0,70257.0,3186.0,2023-01-03 09:15:00,1,9,red,70280.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,70258.0,70313.0,70100.0,70122.0,4688.0,2023-01-03 09:20:00,1,9,red,70178.335938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [7]:
df = ta.add_all_ta_features(
    df,
    open="OPEN",
    high="HIGH",
    low="LOW",
    close="CLOSE",
    volume="VOL",
    fillna=True
)

In [8]:
df["CANDLE_COLOR"] = np.where(
    df["CANDLE_COLOR"] == "green",
    1,
    0
)

In [9]:
df.head()

,OPEN,HIGH,LOW,CLOSE,VOL,DATETIME,WEEK_DAY,HOUR,CANDLE_COLOR,PIVOT,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,70105.0,70500.0,70105.0,70378.0,17203.0,2023-01-03 09:00:00,1,9,1,70327.664062,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,70378.000000,0.000000,0.000000,0.000000
1,70383.0,70392.0,70205.0,70286.0,5659.0,2023-01-03 09:05:00,1,9,0,70294.335938,...,-0.010429,-0.002086,-0.008343,-5.633075,-1.126615,-4.506460,70337.372200,-0.130725,-0.130844,-0.130725
2,70282.0,70360.0,70163.0,70296.0,4963.0,2023-01-03 09:10:00,1,9,1,70273.000000,...,-0.017349,-0.005138,-0.012210,-10.883311,-3.077954,-7.805357,70319.137549,0.014222,0.014210,-0.116515
3,70298.0,70350.0,70233.0,70257.0,3186.0,2023-01-03 09:15:00,1,9,0,70280.000000,...,-0.026996,-0.009510,-0.017486,-16.519141,-5.766192,-10.752949,70291.740017,-0.055480,-0.055504,-0.171930
4,70258.0,70313.0,70100.0,70122.0,4688.0,2023-01-03 09:20:00,1,9,0,70178.335938,...,-0.049565,-0.017521,-0.032044,-20.418678,-8.696689,-11.721989,70216.970447,-0.192153,-0.192261,-0.363749


In [10]:
df["WEEKDAY_SIN"] = sin_transformer(7).fit_transform(df["WEEK_DAY"])
df["WEEKDAY_COS"] = cos_transformer(7).fit_transform(df["WEEK_DAY"])

In [11]:
df["HOUR_SIN"] = sin_transformer(24).fit_transform(df["HOUR"])
df["HOUR_COS"] = cos_transformer(24).fit_transform(df["HOUR"])

In [12]:
df["MINUTE"] = df["DATETIME"].dt.minute

In [13]:
df["MINUTE_SIN"] = sin_transformer(60).fit_transform(df["MINUTE"])
df["MINUTE_COS"] = cos_transformer(60).fit_transform(df["MINUTE"])

In [14]:
df.tail()

,OPEN,HIGH,LOW,CLOSE,VOL,DATETIME,WEEK_DAY,HOUR,CANDLE_COLOR,PIVOT,...,others_dr,others_dlr,others_cr,WEEKDAY_SIN,WEEKDAY_COS,HOUR_SIN,HOUR_COS,MINUTE,MINUTE_SIN,MINUTE_COS
72828,89702.0,89723.0,89695.0,89702.0,257.0,2024-08-30 23:25:00,4,23,0,89706.656250,...,0.004458,0.004482,27.457439,-0.433884,-0.900969,-0.258819,0.965926,25,5.000000e-01,-8.660254e-01
72829,89721.0,89732.0,89701.0,89720.0,292.0,2024-08-30 23:30:00,4,23,0,89717.664062,...,0.020063,0.020123,27.483023,-0.433884,-0.900969,-0.258819,0.965926,30,1.224647e-16,-1.000000e+00
72830,89729.0,89749.0,89710.0,89743.0,218.0,2024-08-30 23:35:00,4,23,1,89734.000000,...,0.025630,0.025558,27.515697,-0.433884,-0.900969,-0.258819,0.965926,35,-5.000000e-01,-8.660254e-01
72831,89727.0,89780.0,89691.0,89699.0,1295.0,2024-08-30 23:40:00,4,23,0,89723.343750,...,-0.049031,-0.049019,27.453184,-0.433884,-0.900969,-0.258819,0.965926,40,-8.660254e-01,-5.000000e-01
72832,89699.0,89775.0,89680.0,89775.0,1481.0,2024-08-30 23:45:00,4,23,1,89743.335938,...,0.084722,0.084686,27.561165,-0.433884,-0.900969,-0.258819,0.965926,45,-1.000000e+00,-1.836970e-16


In [15]:
if "weekday" in df.columns:
    df.drop("weekday", axis=1, inplace=True)

In [16]:
df.set_index(df["DATETIME"], inplace=True)
df.drop("DATETIME", axis=1, inplace=True)

In [17]:
df.head()

,OPEN,HIGH,LOW,CLOSE,VOL,WEEK_DAY,HOUR,CANDLE_COLOR,PIVOT,IS_FRACTAL_UP,...,others_dr,others_dlr,others_cr,WEEKDAY_SIN,WEEKDAY_COS,HOUR_SIN,HOUR_COS,MINUTE,MINUTE_SIN,MINUTE_COS
DATETIME,,,,,,,,,,,,,,,,,,,,,
2023-01-03 09:00:00,70105.0,70500.0,70105.0,70378.0,17203.0,1,9,1,70327.664062,0,...,0.000000,0.000000,0.000000,0.781831,0.62349,0.707107,-0.707107,0,0.000000,1.000000e+00
2023-01-03 09:05:00,70383.0,70392.0,70205.0,70286.0,5659.0,1,9,0,70294.335938,0,...,-0.130725,-0.130844,-0.130725,0.781831,0.62349,0.707107,-0.707107,5,0.500000,8.660254e-01
2023-01-03 09:10:00,70282.0,70360.0,70163.0,70296.0,4963.0,1,9,1,70273.000000,0,...,0.014222,0.014210,-0.116515,0.781831,0.62349,0.707107,-0.707107,10,0.866025,5.000000e-01
2023-01-03 09:15:00,70298.0,70350.0,70233.0,70257.0,3186.0,1,9,0,70280.000000,0,...,-0.055480,-0.055504,-0.171930,0.781831,0.62349,0.707107,-0.707107,15,1.000000,6.123234e-17
2023-01-03 09:20:00,70258.0,70313.0,70100.0,70122.0,4688.0,1,9,0,70178.335938,0,...,-0.192153,-0.192261,-0.363749,0.781831,0.62349,0.707107,-0.707107,20,0.866025,-5.000000e-01


In [18]:
df.drop("TARGET_SHORT", axis=1, inplace=True)

In [22]:
train_df = df.iloc[(
    (df.index >= "2023-02-01") &
    (df.index < "2024-02-01")
)]
valid_df = df.iloc[(
    (df.index >= "2024-02-01") &
    (df.index <= "2024-02-29")
)]
test_df = df.iloc[(
    (df.index >= "2024-03-01") &
    (df.index <= "2024-03-31")
)]

In [23]:
x_train = train_df.drop("TARGET_LONG", axis=1)
y_train = train_df["TARGET_LONG"]
x_valid = valid_df.drop("TARGET_LONG", axis=1)
y_valid = valid_df["TARGET_LONG"]
x_test = test_df.drop("TARGET_LONG", axis=1)
y_test = test_df["TARGET_LONG"]

In [24]:
# grid search
clf = CatBoostClassifier()
params = {'iterations': [500],
          'depth': [4, 5, 6],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10],
#           'eval_metric': ['Accuracy'],
#           'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }
scorer = make_scorer(precision_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

In [25]:
clf_grid.fit(x_train, y_train)
best_param = clf_grid.best_params_
best_param

{'depth': 4,
 'iterations': 500,
 'l2_leaf_reg': 1e-20,
 'leaf_estimation_iterations': 10,
 'logging_level': 'Silent',
 'loss_function': 'Logloss',
 'random_seed': 42}

In [26]:
model = CatBoostClassifier(
    iterations=1000,
    loss_function=best_param["loss_function"],
    depth=best_param["depth"],
    l2_leaf_reg=best_param["l2_leaf_reg"],
    eval_metric="F1",
    leaf_estimation_iterations=10,
    use_best_model=True,
    logging_level=best_param["logging_level"],
    random_seed=42
)
model.fit(x_train, y_train, eval_set=(x_valid, y_valid))

In [27]:
res_valid = model.predict_proba(x_valid)

In [28]:
res_valid = pd.DataFrame(res_valid, columns=["proba_0", "proba_1"])

In [29]:
# подбор уровня отсечения по прибыли
max_res = 0
max_i = 0

for i in range(0, 101, 2):
    res_valid["prediction"] = np.where(res_valid["proba_1"] * 100 >= i, 1, 0)
    res_valid["fin_res"] = np.where(res_valid["prediction"] == 1, 400, -200)
    sum_res = res_valid["fin_res"].sum()
    if sum_res > max_res:
        max_res = sum_res
        max_i = i
    print(f"i: {i} | fin_res: {sum_res}")
print(f"i_max_res: {max_i} | max_fin_res: {max_res}")

i: 0 | fin_res: 1322400
i: 2 | fin_res: 1181400
i: 4 | fin_res: 921000
i: 6 | fin_res: 697200
i: 8 | fin_res: 486000
i: 10 | fin_res: 295800
i: 12 | fin_res: 129000
i: 14 | fin_res: -6600
i: 16 | fin_res: -110400
i: 18 | fin_res: -214200
i: 20 | fin_res: -289800
i: 22 | fin_res: -357600
i: 24 | fin_res: -404400
i: 26 | fin_res: -445800
i: 28 | fin_res: -482400
i: 30 | fin_res: -517200
i: 32 | fin_res: -546000
i: 34 | fin_res: -564000
i: 36 | fin_res: -574800
i: 38 | fin_res: -588600
i: 40 | fin_res: -601800
i: 42 | fin_res: -616200
i: 44 | fin_res: -628200
i: 46 | fin_res: -637200
i: 48 | fin_res: -644400
i: 50 | fin_res: -649200
i: 52 | fin_res: -652800
i: 54 | fin_res: -654600
i: 56 | fin_res: -655200
i: 58 | fin_res: -657000
i: 60 | fin_res: -658200
i: 62 | fin_res: -658800
i: 64 | fin_res: -660000
i: 66 | fin_res: -660600
i: 68 | fin_res: -661200
i: 70 | fin_res: -661200
i: 72 | fin_res: -661200
i: 74 | fin_res: -661200
i: 76 | fin_res: -661200
i: 78 | fin_res: -661200
i: 80 | fin_

In [30]:
# метрики качества
for i in range(0, 101, 2):
    res_valid["prediction"] = np.where(res_valid["proba_1"] * 100 >= i, 1, 0)
    precision = precision_score(y_valid, res_valid["prediction"])
    recall = recall_score(y_valid, res_valid["prediction"])
    f1 = f1_score(y_valid, res_valid["prediction"])
    print(f"i: {i} | precision: {precision} | recall: {recall} | F1: {f1}")

i: 0 | precision: 0.3759830611010284 | recall: 1.0 | F1: 0.5464937348867883
i: 2 | precision: 0.38782155649625527 | recall: 0.9581657280772325 | F1: 0.5521557719054242
i: 4 | precision: 0.4141069397042093 | recall: 0.8785197103781175 | F1: 0.5628865979381443
i: 6 | precision: 0.4297703180212014 | recall: 0.7827835880933226 | F1: 0.5548902195608783
i: 8 | precision: 0.4524058577405858 | recall: 0.6958970233306516 | F1: 0.5483359746434231
i: 10 | precision: 0.470846394984326 | recall: 0.6041834271922768 | F1: 0.529245947850599
i: 12 | precision: 0.4806378132118451 | recall: 0.5092518101367659 | F1: 0.49453125
i: 14 | precision: 0.5132905591200734 | recall: 0.4505229283990346 | F1: 0.47986289631533846
i: 16 | precision: 0.5413943355119826 | recall: 0.3998390989541432 | F1: 0.45997223507635354
i: 18 | precision: 0.5691275167785235 | recall: 0.3411102172164119 | F1: 0.42655935613682094
i: 20 | precision: 0.5977382875605816 | recall: 0.29766693483507645 | F1: 0.39742212674543503
i: 22 | prec

In [31]:
pred_test = model.predict_proba(x_test)

In [32]:
pred_test = pd.DataFrame(pred_test, columns=["proba_0", "proba_1"])

In [33]:
pred_test["prediction"] = np.where(pred_test['proba_1'] >= 0.04, 1, 0)
pred_test["fin_res"] = np.where(pred_test["prediction"] == 1, 400, -200)

In [34]:
prec_test = precision_score(y_test, pred_test["prediction"])
rec_test = recall_score(y_test, pred_test["prediction"])
f1_test = f1_score(y_test, pred_test["prediction"])
print(f"precision: {prec_test} | recall: {rec_test} | f1: {f1_test}")

precision: 0.4061291282356442 | recall: 0.9956236323851203 | f1: 0.5769230769230769


In [35]:
print(f"Fin res: {pred_test['fin_res'].sum()}")

Fin res: 1320600


In [36]:
x_test.shape

(3480, 139)

In [37]:
df_test = x_test.copy()

In [38]:
df_test["TARGET_LONG"] = y_test

In [39]:
pred_test

,proba_0,proba_1,prediction,fin_res
0,0.833594,0.166406,1,400
1,0.860916,0.139084,1,400
2,0.895294,0.104706,1,400
3,0.883797,0.116203,1,400
4,0.867537,0.132463,1,400
...,...,...,...,...
3475,0.856436,0.143564,1,400
3476,0.908397,0.091603,1,400
3477,0.884728,0.115272,1,400
3478,0.819803,0.180197,1,400


In [40]:
df_test["PREDICTION_PROBA"] = pred_test["proba_1"].values
df_test["PREDICTION"] = pred_test["prediction"].values

In [41]:
df_test

,OPEN,HIGH,LOW,CLOSE,VOL,WEEK_DAY,HOUR,CANDLE_COLOR,PIVOT,IS_FRACTAL_UP,...,WEEKDAY_SIN,WEEKDAY_COS,HOUR_SIN,HOUR_COS,MINUTE,MINUTE_SIN,MINUTE_COS,TARGET_LONG,PREDICTION_PROBA,PREDICTION
DATETIME,,,,,,,,,,,,,,,,,,,,,
2024-03-01 09:00:00,91628.0,91750.0,91628.0,91681.0,6242.0,4,9,1,91686.335938,1,...,-0.433884,-0.900969,0.707107,-0.707107,0,0.000000e+00,1.000000e+00,1,0.166406,1
2024-03-01 09:05:00,91681.0,91704.0,91640.0,91699.0,2114.0,4,9,1,91681.000000,0,...,-0.433884,-0.900969,0.707107,-0.707107,5,5.000000e-01,8.660254e-01,1,0.139084,1
2024-03-01 09:10:00,91699.0,91740.0,91699.0,91714.0,2338.0,4,9,1,91717.664062,1,...,-0.433884,-0.900969,0.707107,-0.707107,10,8.660254e-01,5.000000e-01,1,0.104706,1
2024-03-01 09:15:00,91715.0,91726.0,91680.0,91680.0,2588.0,4,9,0,91695.335938,0,...,-0.433884,-0.900969,0.707107,-0.707107,15,1.000000e+00,6.123234e-17,1,0.116203,1
2024-03-01 09:20:00,91680.0,91725.0,91670.0,91720.0,1730.0,4,9,1,91704.992188,0,...,-0.433884,-0.900969,0.707107,-0.707107,20,8.660254e-01,-5.000000e-01,1,0.132463,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29 23:25:00,94470.0,94471.0,94461.0,94471.0,173.0,4,23,1,94467.664062,0,...,-0.433884,-0.900969,-0.258819,0.965926,25,5.000000e-01,-8.660254e-01,0,0.143564,1
2024-03-29 23:30:00,94465.0,94493.0,94461.0,94477.0,202.0,4,23,1,94477.007812,1,...,-0.433884,-0.900969,-0.258819,0.965926,30,1.224647e-16,-1.000000e+00,0,0.091603,1
2024-03-29 23:35:00,94482.0,94482.0,94465.0,94465.0,96.0,4,23,0,94470.664062,0,...,-0.433884,-0.900969,-0.258819,0.965926,35,-5.000000e-01,-8.660254e-01,0,0.115272,1


In [42]:
df_test.to_csv(
    TEST_SAVE_NAME,
    sep=";",
    encoding="cp1251",
    index=False
)